# Projecting data on a map

In this example, we will see how to use xsar to project data on a map, or to export to geotiff.

In [ ]:
import xsar
import xarray as xr
import numpy as np
import geoviews as gv
import holoviews as hv
xr.set_options(display_expand_data=False, display_expand_attrs=False)
gv.extension('bokeh')
hv.extension('bokeh')


## Opening the dataset

When using projection, it's mandatory to open the dataset with [xsar.Sentinel1Dataset](../basic_api.rst#xsar.Sentinel1Dataset).

In [ ]:
filename = xsar.get_test_file('S1B_IW_GRDH_1SDV_20181013T062322_20181013T062347_013130_018428_Z010.SAFE')
s1meta = xsar.Sentinel1Meta(filename)
xsar_obj = xsar.Sentinel1Dataset(s1meta, resolution='100m')

## Display the image without projection

When using `holoviews.Image` the data is displayed as the memory order

In [ ]:
hv.Image(xsar_obj.dataset.sigma0.sel(pol='VV')).opts(alpha=0.7, cmap='gray', clim=(0,0.05))

To have it displayed the same way it was aquired, we need to pass `kdims=['sample', 'line']`.

We can see here that the orbit pass is **descending**

In [ ]:
hv.Image(xsar_obj.dataset.sigma0.sel(pol='VV'), kdims=['sample', 'line']).opts(alpha=0.7, cmap='gray', clim=(0,0.05))

## Reprojecting to epsg 4326, using rioxarray.reproject

Reprojection are done with the `rioxarray` [.rio](https://corteva.github.io/rioxarray/html/rioxarray.html#rioxarray-rio-accessors) accessor. 

Attributes accessor are volatile (see [Managing Information Loss with xarray operations](https://corteva.github.io/rioxarray/stable/getting_started/manage_information_loss.html)), so it's a good practice to call [xsar.BaseDataset.recompute_attrs()](../basic_api.rst#xsar.BaseDataset.recompute_attrs) before using it.


'epsg:4326' is the standard lon/lat projection, in degrees. 


In [ ]:
import rasterio
xsar_obj.recompute_attrs()
sigma0_proj = xsar_obj.dataset['sigma0'].rio.reproject('epsg:4326',shape=(1000,1000),nodata=np.nan)
sigma0_proj

The `sigma0_proj` dataarray is georeferenced, so holoviews know how to deal with it.

In [ ]:
sigma0_image = gv.Image(sigma0_proj.sel(pol='VV')).opts(alpha=0.3, cmap='gray', clim=(0,0.05))
#(gv.tile_sources.Wikipedia * gv.feature.land.options(scale='50m') * sigma0_image).opts(width=600,height=600)
(gv.tile_sources.EsriImagery * sigma0_image * gv.feature.coastline.options(scale='10m')).opts(width=600,height=600)

## Exporting to geotiff

### Exporting without colormap

using [rioxarray.raster_array.RasterArray.to_raster](https://corteva.github.io/rioxarray/html/rioxarray.html#rioxarray.raster_array.RasterArray.to_raster), we can save our projected sigma0 to a geotiff

In [ ]:
s0vvproj = sigma0_proj.sel(pol='VV')
s0vvproj.attrs['name'] = 'sigma0 VV projected using rioxarray'
s0vvproj.rio.to_raster('/tmp/sigma0_nocolor.tiff')

The geotiff can be read with `rioxarray` via `rasterio` but it's stored as float, and we have to pass a cmap. This image cannot be viewed with google earth 

In [ ]:
#gv.tile_sources.Wikipedia * gv.load_tiff('/tmp/sigma0_nocolor.tiff').opts(alpha=0.7, cmap='gray', clim=(0,0.05))
import rioxarray
tmp = rioxarray.open_rasterio('/tmp/sigma0_nocolor.tiff')
tmp = tmp.rename('sigma0_nocolor')
tmp2 = gv.util.from_xarray(tmp)
gv.tile_sources.EsriImagery * gv.Image(tmp2,kdims=['x','y']).opts(alpha=0.7, cmap='gray', clim=(0,0.05),colorbar=True,width=500,height=200) # vdims='sigma0_nocolor'

### Exporting with RGBA colormap

We have to manually convert the sigma0 floating values to `np.uint8` in the range [0,255]

In [ ]:
from matplotlib import cm
cmap=cm.ScalarMappable(cmap='jet')
cmap.set_clim(vmin=0, vmax=0.05)
rgb_sigma0 = xr.DataArray(
    (cmap.to_rgba(xsar_obj.dataset['sigma0'].sel(pol='VV'), bytes=True)),  
    dims=['line', 'sample', 'band']
).transpose('band', 'line', 'sample')
rgb_sigma0


`rgb_sigma0` is now an `xarray.DataArray`, with sames spatials dims `['line', 'sample']`, and an new dim `band` that hold color in R,G,B,A. 

This dataarray is currently not georeferenced. To georefence it, we have **to store it into xsar_obj.dataset and call [xsar.BaseDataset.recompute_attrs()](../basic_api.rst#xsar.BaseDataset.recompute_attrs)**.


In [ ]:
#xsar_obj.dataset['sigma0_rgba'] = rgb_sigma0 
rgb_sigma0.name = 'sigma0_rgba'
xsar_obj.datatree['measurement'] = xsar_obj.datatree['measurement'].assign(xr.merge([xsar_obj.dataset,rgb_sigma0]))
xsar_obj.recompute_attrs()
s0rgba = xsar_obj.dataset['sigma0_rgba'].rio.reproject('epsg:4326',shape=(1000,1000),nodata=0)
s0rgba.attrs['name'] = 'sigma0 RGBA projected using rioxarray'
s0rgba.rio.to_raster('/tmp/sigma0_color.tiff')

In [ ]:
# there is a transparency bug in geoviews (https://github.com/holoviz/geoviews/issues/571)
# but if loading this tiff in google earth, it should render properly
tmp = rioxarray.open_rasterio('/tmp/sigma0_color.tiff')
tmp = tmp.rename('sigma0_color').isel(band=1)
print(tmp)
# tmp2 = gv.util.from_xarray(tmp,vdims='sigma0_color')
# print('empt2\n====================',tmp2)
# (gv.tile_sources.Wikipedia * gv.load_tiff('/tmp/sigma0_color.tiff')).opts(width=600,height=600)
(gv.tile_sources.EsriImagery * gv.Image(tmp,vdims='sigma0_color',kdims=['x','y']).opts(cmap='magma',colorbar=True,tools=['hover'])).opts(width=600,height=600) # 

## Reprojecting to pre-defined grid

We now want to project ecmwf data and sigma0 data to the same grid.

The grid is choosen to be 600km*600km , with a spacing of 1km. x and y coords are in **meters**, because we are going to use an **azimuthal equidistant projection**, centered on the xsar dataset

In [ ]:
from rasterio.crs import CRS
from xsar.raster_readers import ecmwf_0100_1h

grid = xr.DataArray(
    np.full((601,601), np.nan), 
    dims=['x','y'], 
    coords={
        'x': np.linspace(-300 * 1000,300 * 1000, 601), 
        'y': np.linspace(-300 * 1000,300 * 1000, 601)
    })


crs = CRS({
        'proj': 'aeqd',
        'lat_0': xsar_obj.sar_meta.footprint.centroid.y,
        'lon_0': xsar_obj.sar_meta.footprint.centroid.x,
        'x_0': 0,
        'y_0': 0,
        'ellps': 'WGS84'
    })

grid.rio.write_crs(crs, inplace=True)
grid

Now, we can use [reproject_match](https://corteva.github.io/rioxarray/stable/rioxarray.html#rioxarray.raster_array.RasterArray.reproject_match), to project onto `grid`

In [ ]:

xsar_obj.recompute_attrs()
sigma0_grid = xsar_obj.dataset['sigma0'].rio.reproject_match(grid,nodata=np.nan).sel(pol='VV')
ecmwf_ds = ecmwf_0100_1h(xsar.get_test_file('ECMWF_FORECAST_0100_202109091300_10U_10V.nc'))
ecmwf_ds['spd'] = np.sqrt(ecmwf_ds['U10']**2+ecmwf_ds['V10']**2)
ecmwf_grid = ecmwf_ds['spd'].rio.reproject_match(grid,nodata=np.nan)

# ecmwf_grid and sigma0_grid has the same shape and the same coordinates, we are able to merge them

merged_grid = xr.merge([sigma0_grid, ecmwf_grid])

geoviews and cartopy are not able to handle azimuthal equidistant projection, so we use `holoviews` here.

(But note that it will display properly in google earth, if exported to geotiff)

In [ ]:
import holoviews as hv
hv.extension('bokeh')
hv.Image(merged_grid['spd']).opts(cmap='jet') * hv.Image(merged_grid['sigma0']).opts(cmap='gray', clim=(0,0.05), tools=['hover']) 

To use holoviews, we can reproject to lon/lat

In [ ]:
merged_lonlat = merged_grid.rio.reproject(4326)
(
    gv.tile_sources.EsriImagery * gv.Image(merged_lonlat['spd']).opts(cmap='jet', alpha=0.3) 
    * gv.Image(merged_lonlat['sigma0']).opts(cmap='gray', clim=(0,0.05), alpha=0.7)
).opts(width=600,height=600)
                                                           

## Mapping a raster onto original xsar grid

Using `rioxarray.reproject_match` with a destination grid containing gcps (like xsar dataset) is currently not supported, but xsar provide [xsar.BaseDataset.map_raster](../basic_api.rst#xsar.BaseDataset.map_raster).

It's argument is a projected dataset, that we want to map onto the xsar grid.



In [ ]:
#xsar_obj.dataset['sigma0_rasterized'] = 
tmp_ds = xsar_obj.map_raster(merged_grid['sigma0'])
tmp_ds.name = 'sigma0_rasterized'
xsar_obj.datatree['measurement'] = xsar_obj.datatree['measurement'].assign(xr.merge([xsar_obj.dataset,tmp_ds]))
xsar_obj.recompute_attrs()

In [ ]:
(
    hv.Image(xsar_obj.dataset['sigma0_rasterized'], kdims=['sample','line']).opts(title='from raster') 
    + hv.Image(xsar_obj.dataset.sigma0.sel(pol='VV'), kdims=['sample','line']).opts(title='original') 
).opts(hv.opts.Image(tools=['hover'],cmap='gray', clim=(0,0.05), alpha=0.7))